In [2]:
import json
import pandas as pd
from owlready2 import *
import re
import fasttext 
import numpy as np
from numpy import dot
from numpy.linalg import norm
import networkx as nx
import annoy

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
AITopLevelTopics = ['Artificial intelligence', 'Computer vision', 'Data mining',
                     'Data science', 'Machine learning', 'Natural language processing',
                     'Pattern recognition', 'Speech recognition']
with open("dblpPaperIDs.json", 'r') as f:
    paperList = json.load(f)
papersUnderConsideration = set(paperList)

In [ ]:
paperData = dict()
with open('dblp_AIpapers_v11.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        
        dataDict = dict()
        dataDict['title'] = data.get('title', '')
        references = list()
        for reference in data.get('references',[]):
            if reference in papersUnderConsideration:
                references.append(reference)
        dataDict['references'] = references
        dataDict['abstract'] = []
        if 'indexed_abstract' in data:
            dataDict['abstract'] = [w for w in data['indexed_abstract']['InvertedIndex'].keys() if len(w) > 1]                    
        dataDict['fos'] = data.get('fos',[])
        # paperData[paperID] = dataDict
        tmpDict = dataDict.copy()
        tmpDict['id'] = paperID
        
        paperID = data['id'] 
            
            


In [7]:
fasttextModel = fasttext.load_model('crawl-300d-2M-subword.bin')

In [19]:
with open('dblp_AIpapers_v11.json', 'r') as file:
    with open('dblpAbstractFTEmbeddings.json', 'w') as outfile:
        for line in file:
            data = json.loads(line)
            paperID = data['id'] 
            abstractWordList = data.get('abstract',[])
            abstractString = ' '.join(word for word in abstractWordList)
            abstractString = abstractString.replace('\n', ' ').replace('\r', '')
            embedding = fasttextModel.get_sentence_vector(abstractString).tolist()    # while reading use np.asarray to convert to np array
            outDict = dict()
            outDict['id'] = paperID
            outDict['embedding'] = embedding
            json.dump(outDict, outfile)
            outfile.write('\n')

In [ ]:
!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC ./module/UnivTransformer

In [4]:
module_url = "./module/UnivTrans" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
def embed(inputText):
    return model(inputText)

In [21]:
abstractList = []
paperIDList = []
with open('dblp_AIpapers_v11.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        paperID = data['id'] 
        abstractWordList = data.get('abstract',[])
        abstractString = ' '.join(word for word in abstractWordList)
        abstractString = abstractString.replace('\n', ' ').replace('\r', '')
        paperIDList.append(paperID)
        abstractList.append(abstractString)

In [ ]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    abstractEmbeddings = session.run(embed(abstractList))
del abstractList[:] 

In [ ]:
with open('dblpAbstractUSETransEmbeddings.json', 'w') as outfile:
    for paperId, embedding in zip(paperIDList, abstractEmbeddings):
        outDict = dict()
        outDict['id'] = paperID
        outDict['embedding'] = embedding.tolist()
        json.dump(outDict, outfile)
        outfile.write('\n')

In [19]:
del messages[:]

In [20]:
messages

[]